## Predicting pIC50 for JAK2 with RNNs and SMILES strings 

This notebook demonstrates how to build predictive recurrent neural network for SMILES strings. We will build classification model for logP with OpenChem Toolkit (https://github.com/Mariewelt/OpenChem)

In [1]:
# Cloning OpenChem. Comment this line if you already cloned the repository
 ! git clone https://github.com/Mariewelt/OpenChem.git

## Imports

In [1]:
import sys
import os

In [2]:
sys.path.append('../OpenChem')

In [3]:
from openchem.models.Smiles2Label import Smiles2Label
from openchem.modules.encoders.rnn_encoder import RNNEncoder
from openchem.modules.mlp.openchem_mlp import OpenChemMLP
from openchem.data.smiles_data_layer import SmilesDataset
from openchem.data.utils import save_smiles_property_file
from openchem.data.utils import create_loader
from openchem.models.openchem_model import build_training, fit, evaluate

C:\Users\U047856\Programs\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
import torch.nn as nn
from torch.optim import RMSprop, SGD, Adam
from torch.optim.lr_scheduler import ExponentialLR, StepLR
import torch.nn.functional as F

In [5]:
from sklearn.metrics import roc_auc_score
import pandas as pd
import copy
import pickle

## Reading data

In [6]:
from openchem.data.utils import read_smiles_property_file
data = read_smiles_property_file('./data/jak2_data.csv', 
                                 cols_to_read=[0, 1])
smiles = data[0]
labels = data[1].astype('float32')

In [7]:
import numpy as np
binary_labels = (labels >= 7.0).astype('float32').reshape(-1)

In [8]:
from openchem.data.utils import get_tokens
tokens, _, _ = get_tokens(smiles)
tokens = tokens + ' '

## Model architecture

Here we define the architecture of our Recurrent Neural Network (RNN). We will use 2 LSTM layers. For more details on how to build models with OpenChem, visit: https://mariewelt.github.io/OpenChem/

In [19]:
import torch
from openchem.utils.utils import identity
from openchem.modules.embeddings.basic_embedding import Embedding
model_object = Smiles2Label

model_params = {
    'use_cuda': True,
    'random_seed': 42,
    'world_size': 1,
    'task': 'classification',
    'data_layer': SmilesDataset,
    'use_clip_grad': False,
    'batch_size': 128,
    'num_epochs': 51,
    'logdir': './logs/jak2_logs',
    'print_every': 1,
    'save_every': 5,
    #'train_data_layer': train_dataset,
    #'val_data_layer': test_dataset,
    'eval_metrics': roc_auc_score,
    'criterion': nn.CrossEntropyLoss(),
    'optimizer': Adam,
    'optimizer_params': {
        'lr': 0.005,
    },
    'lr_scheduler': StepLR,
    'lr_scheduler_params': {
        'step_size': 15,
        'gamma': 0.8
    },
    'embedding': Embedding,
    'embedding_params': {
        'num_embeddings': len(tokens),
        'embedding_dim': 128,
        'padding_idx': tokens.index(' ')
    },
    'encoder': RNNEncoder,
    'encoder_params': {
        'input_size': 128,
        'layer': "LSTM",
        'encoder_dim': 128,
        'n_layers': 2,
        'dropout': 0.8,
        'is_bidirectional': False
    },
    'mlp': OpenChemMLP,
    'mlp_params': {
        'input_size': 128,
        'n_layers': 2,
        'hidden_size': [128, 2],
        'activation': F.relu,
        'dropout': 0.0
    }
}

In [20]:
try:
    os.stat(model_params['logdir'])
except:
    os.mkdir(model_params['logdir'])

## Initializing data splitter for cross validation

In [21]:
from sklearn.model_selection import KFold
cross_validation_split = KFold(n_splits=5, shuffle=True)

In [22]:
data = cross_validation_split.split(smiles, binary_labels)

## Training cross-validated models

In [23]:
import os
i = 0
models = []
results = []
for split in data:
    print('Cross validation, fold number ' + str(i) + ' in progress...')
    train, test = split
    X_train = smiles[train]
    y_train = binary_labels[train].reshape(-1)
    X_test = smiles[test]
    y_test = binary_labels[test].reshape(-1)
    save_smiles_property_file('./data/jak2_train_fold' + str(i) + '.smi', 
                              X_train, y_train.reshape(-1, 1))
    save_smiles_property_file('./data/jak2_test_fold' + str(i) + '.smi', 
                              X_test, y_test.reshape(-1, 1))

    train_dataset = SmilesDataset('./data/jak2_train_fold' + str(i) + '.smi',
                           delimiter=',', cols_to_read=[0, 1], tokens=tokens)
    train_dataset.target = train_dataset.target.reshape(-1)
    test_dataset = SmilesDataset('./data/jak2_test_fold' + str(i) + '.smi',
                       delimiter=',', cols_to_read=[0, 1], tokens=tokens)
    test_dataset.target = test_dataset.target.reshape(-1)
    model_params['train_data_layer'] = train_dataset
    model_params['val_data_layer'] = test_dataset
    model_params['logdir'] = './logs/jak2_logs/fold' + str(i)  
    logdir = model_params['logdir']
    ckpt_dir = logdir + '/checkpoint/'
    try:
        os.stat(ckpt_dir)
    except:
        os.mkdir(logdir)
        os.mkdir(ckpt_dir)
    train_loader = create_loader(train_dataset,
                             batch_size=model_params['batch_size'],
                             shuffle=True,
                             num_workers=4,
                             pin_memory=True,
                             sampler=None)
    val_loader = create_loader(test_dataset,
                           batch_size=model_params['batch_size'],
                           shuffle=False,
                           num_workers=1,
                           pin_memory=True)
    models.append(model_object(params=model_params).cuda())
    criterion, optimizer, lr_scheduler = build_training(models[i], model_params)
    results.append(fit(models[i], lr_scheduler, train_loader, optimizer, criterion,
        model_params, eval=True, val_loader=val_loader))
    
    i = i+1

Cross validation, fold number 0 in progress...


RuntimeError: Cannot initialize CUDA without ATen_cuda library. PyTorch splits its backend into two shared libraries: a CPU library and a CUDA library; this error has occurred because you are trying to use some CUDA functionality, but the CUDA library has not been loaded by the dynamic linker for some reason.  The CUDA library MUST be loaded, EVEN IF you don't directly use any symbols from the CUDA library! One common culprit is a lack of -Wl,--no-as-needed in your link arguments; many dynamic linkers will delete dynamic library dependencies if you don't depend on any of their symbols.  You can check if this has occurred by using ldd on your binary to see if there is a dependency on *_cuda.so library.

## Evaluating the models

In [18]:
import numpy as np
rmse = []
auc_score = []
for i in range(5):
    test_dataset = SmilesDataset('./data/jak2_test_fold' + str(i) + '.smi',
                                 delimiter=',', cols_to_read=[0, 1], tokens=tokens)
    test_dataset.target = test_dataset.target.reshape(-1)
    val_loader = create_loader(test_dataset,
                               batch_size=model_params['batch_size'],
                               shuffle=False,
                               num_workers=1,
                               pin_memory=True)
    metrics = evaluate(models[i], val_loader, criterion)
    rmse.append(np.sqrt(metrics[0]))
    auc_score.append(metrics[1])

EVALUATION: [Time: 0m 0s, Loss: 0.5376, Metrics: 0.7529]
EVALUATION: [Time: 0m 0s, Loss: 0.5504, Metrics: 0.7556]
EVALUATION: [Time: 0m 0s, Loss: 0.5522, Metrics: 0.7740]
EVALUATION: [Time: 0m 0s, Loss: 0.5435, Metrics: 0.7549]
EVALUATION: [Time: 0m 0s, Loss: 0.5699, Metrics: 0.7641]


In [19]:
print("Cross-validated RMSE: ",  np.mean(rmse))
print("Cross-validated AUC score: ", np.mean(auc_score))

Cross-validated RMSE:  0.742072568424805
Cross-validated AUC score:  0.7603019075230668
